<a href="https://colab.research.google.com/github/DininduChamikara/Simple-Transformers-NER-Sinhala/blob/main/Simple_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import pandas as pd
data = pd.read_csv("/content/gdrive/MyDrive/Research Project/ner_dataset_v3.0.csv", encoding="utf-8")
data.head()

,Sentence #,Word,Tag
0,1,ආසියානු,LOCATION
1,1,ක්‍රිකට්,OTHER
2,1,ශුර,OTHER
3,1,ශ්‍රී,LOCATION
4,1,ලංකා,LOCATION


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
data.rename(columns={"Sentence #": "sentence_id", "Word": "words", "Tag": "labels"}, inplace=True)

In [ ]:
data

,sentence_id,words,labels
0,1,ආසියානු,LOCATION
1,1,ක්‍රිකට්,OTHER
2,1,ශුර,OTHER
3,1,ශ්‍රී,LOCATION
4,1,ලංකා,LOCATION
...,...,...,...
23608,713,ප්‍රවාහනය,OTHER
23609,713,කිරීමත්,OTHER
23610,713,තහනම්,OTHER
23611,713,කර,OTHER


In [ ]:
X = data[["sentence_id", "words"]]
Y = data["labels"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
# building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"], "words":x_train["words"], "labels":y_train})
test_data = pd.DataFrame({"sentence_id": x_test["sentence_id"], "words": x_test["words"], "labels": y_test})

In [ ]:
train_data

,sentence_id,words,labels
13147,460,නොකී,OTHER
15430,514,පාහේ,OTHER
845,40,ගෙදරින්,OTHER
9015,326,බව,OTHER
13181,462,හැටි,OTHER
...,...,...,...
6326,262,‘‘උදකස්දා’’,OTHER
10029,354,කහ,OTHER
6251,260,තිබෙනවා.,OTHER
15368,513,සඳහනක්,OTHER


###Model Training

In [ ]:
from simpletransformers.ner import NERModel, NERArgs

In [ ]:
label = data['labels'].unique().tolist()
label

['LOCATION', 'OTHER', 'PERSON', 'DATE', 'ORGANIZATION', 'TIME']

In [ ]:
args = NERArgs()
args.num_train_epochs = 5
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

In [ ]:
model = NERModel('bert', 'bert-base-cased', labels=label, args=args, use_cuda= False)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.train_model(train_data, eval_data=test_data, acc=accuracy_score)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/23 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/23 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/23 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/23 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/23 [00:00<?, ?it/s]

(115, 0.5168189935062243)

In [21]:
result, model_outputs, preds_list = model.eval_model(test_data)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/21 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOCATION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PERSON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ORGANIZATION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DATE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_

In [22]:
result

{'eval_loss': 0.48781322723343257,
 'precision': 0.8115942028985508,
 'recall': 0.1346153846153846,
 'f1_score': 0.2309278350515464}

In [23]:
prediction, model_output = model.predict(["2023-2025 ලෝක ටෙස්ට් ශූරතා තරගාවලිය එළඹෙන ජූනි මාසයේ දී ආරම්භ වීමට නියමිත යි. ඒ සඳහා සහභාගි වන එක් එක් කණ්ඩායමට හිමි වන ටෙස්ට් තරග ප්‍රමාණය මේ වන විට ප්‍රකාශයට පත් කර අවසන්. ඒ අනුව මෙවර තරගාවලිය යටතේ ද ශ්‍රී ලංකාවට ටෙස්ට් තරග 12කට ක්‍රීඩා කිරීමේ අවස්ථාව ලැබෙනවා. එයින් ටෙස්ට් තරග 6ක් ශ්‍රී ලංකාවේ ද ඉතිරි ටෙස්ට් තරග 6 විදෙස් රටවල්වල දී ද පැවැත්වීමට නියමිත යි.අවසන් වරට පැවති 2021-2023 ලෝක ටෙස්ට් ශූරතා තරගාවලියේ දී ශ්‍රී ලංකාව සැලකිය යුතු තරම් දස්කම් පෙන්වීමට සමත් වුණා. එනිසා මෙවර ලෝක ටෙස්ට් ශූරතාවේ අවසන් තරගය සඳහා පිවිසීමට ශ්‍රී ලංකාවට තිබෙන අවස්ථාව පිළිබඳව යම් විශ්ලේෂණයක් මෙලෙසින් ඔබ වෙත ගෙන එනවා."])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
prediction

[[{'2023-2025': 'DATE'},
  {'ලෝක': 'OTHER'},
  {'ටෙස්ට්': 'OTHER'},
  {'ශූරතා': 'OTHER'},
  {'තරගාවලිය': 'OTHER'},
  {'එළඹෙන': 'OTHER'},
  {'ජූනි': 'OTHER'},
  {'මාසයේ': 'OTHER'},
  {'දී': 'OTHER'},
  {'ආරම්භ': 'OTHER'},
  {'වීමට': 'OTHER'},
  {'නියමිත': 'OTHER'},
  {'යි.': 'OTHER'},
  {'ඒ': 'OTHER'},
  {'සඳහා': 'OTHER'},
  {'සහභාගි': 'OTHER'},
  {'වන': 'OTHER'},
  {'එක්': 'OTHER'},
  {'එක්': 'OTHER'},
  {'කණ්ඩායමට': 'OTHER'},
  {'හිමි': 'OTHER'},
  {'වන': 'OTHER'},
  {'ටෙස්ට්': 'OTHER'},
  {'තරග': 'OTHER'},
  {'ප්\u200dරමාණය': 'OTHER'},
  {'මේ': 'OTHER'},
  {'වන': 'OTHER'},
  {'විට': 'OTHER'},
  {'ප්\u200dරකාශයට': 'OTHER'},
  {'පත්': 'OTHER'},
  {'කර': 'OTHER'},
  {'අවසන්.': 'OTHER'},
  {'ඒ': 'OTHER'},
  {'අනුව': 'OTHER'},
  {'මෙවර': 'OTHER'},
  {'තරගාවලිය': 'OTHER'},
  {'යටතේ': 'OTHER'},
  {'ද': 'OTHER'},
  {'ශ්\u200dරී': 'OTHER'},
  {'ලංකාවට': 'OTHER'},
  {'ටෙස්ට්': 'OTHER'},
  {'තරග': 'OTHER'},
  {'12කට': 'OTHER'},
  {'ක්\u200dරීඩා': 'OTHER'},
  {'කිරීමේ': 'OTHER'},
  {'අවස්ථාව': '